In [1]:
import sys
sys.path.append('../cfq')

from itertools import *
import pickle
import numpy as np
from data import RaggedArray, CFQDataset

np.set_printoptions(linewidth=150)

Using backend: pytorch


In [2]:
class bcolors:
    R = '\033[31m'
    G = '\033[32m'
    Y = '\033[33m'
    B = '\033[34m'
    ENDC = '\033[0m'

In [3]:
input_dir = '/work/yu_gai/cfq/data/cfq'

In [4]:
df = sqlCtx.read.parquet(f'/data/yu_gai/cfq/dataset.parquet/').sort('index').persist()
question = df.rdd.map(lambda r: r['questionPatternModEntities']).collect()
template = df.rdd.map(lambda r: r['questionTemplate']).collect()
sparql = df.rdd.map(lambda r: r['sparqlPatternModEntities']).collect()

In [142]:
dat = np.load(f'{input_dir}/data.npz')
idx = np.load(f'{input_dir}/splits/mcd3.npz')['testIdxs']
tok_vocab = idx2tok, _ = pickle.load(open(f'{input_dir}/tok-vocab.pickle', 'rb'))
tag_vocab = idx2tag, tag2idx = pickle.load(open(f'{input_dir}/tag-vocab.pickle', 'rb'))
typ_vocab = idx2typ, _ = pickle.load(open(f'{input_dir}/typ-vocab.pickle', 'rb'))
dataset = CFQDataset(idx, dat, tok_vocab, tag_vocab, typ_vocab)
getitem = lambda key: dataset[np.where(idx == key)[0].item()]

In [186]:
# out_dir = '/work/yu_gai/cfq/cfq/../data/runs/None_012521_202944'
out_dir = '/work/yu_gai/cfq/cfq/../data/runs/None_012521_204208'
out = np.load(f'{out_dir}/test_dict.npz')

In [187]:
n = np.array([getitem(index)['n'] for index in out['index']])
m = n * n
indptr = np.hstack([np.zeros(1), m.cumsum()]).astype(int)

In [188]:
def pos_typ(mem, n_mem, src, dst, typ):
    mem = RaggedArray(mem, np.cumsum(np.hstack([[0], n_mem])))
    i, j = np.where(typ)
    rels = []
    for u, t, v in zip(src[i], j, dst[i]):
        for x, y in product(mem[u], mem[v]):
            rels.append(f'{idx2tok[y]} {idx2typ[t]} {idx2tok[x]}')
    print(*sorted(rels), sep='\n')

In [189]:
seq2str = lambda seq: ' '.join(idx2tag[idx] for idx in seq)

In [190]:
out['em'].mean()

0.5026737967914439

In [192]:
np.set_printoptions(precision=1, linewidth=500)

In [209]:
idx2tag.extend(['NP', 'vars'])
for em, index, start, end in islice(zip(out['em'], out['index'], indptr[:-1], indptr[1:]), 100):
    if True:
        dat = getitem(index)
        
        print(bcolors.B if em else bcolors.R, question[index], bcolors.ENDC)
#         print(' '.join(idx2tag[idx] for idx in dat['seq']))
        seq_tag = ' '.join(idx2tag[idx] for idx in dat['seq_tag'])
#         if seq_tag != 'Did NP [VP_SIMPLE] NP {NIL}':
#             continue
        print(seq_tag)
        for seq_noun in dat['seq_noun']:
            print(' '.join(idx2tag[idx] for idx in seq_noun))
            
        print('*' * 75)
        
        isrel = lambda r: all(s not in r for s in ['!=', ' a ', 'ns:people.person.gender', 'ns:people.person.nationality'])
        print(*sorted(filter(isrel, sparql[index].split('\n')[1 : -1])), sep='\n')
#         print(dat['pos_noun'], dat['pos_tag'], dat['pos_np'], dat['pos_nps'])
#         print(dat['isnp'], dat['pos_nps'], dat['pos_tag'])
        
#         templ = ' '.join([idx2tag[idx] for idx in dat['seq']])
        src, dst = dat['src'], dat['dst']
        typ_true = out['typ_true'][start : end]
        typ_pred = out['typ_pred'][start : end]
        logit = out['logit'][start : end]
        
#         print(src, dst)
#         print(typ_true.astype(int))
#         print(typ_pred.astype(int))
        
        mem = np.hstack([dat['mem'], dat['seq_var']])
        n_mem = np.hstack([dat['n_mem'], np.ones(len(dat['seq_var']), dtype=int)])
        print('*' * 75)
        pos_typ(mem, n_mem, src, dst, typ_true)
        
        print('*' * 75)
        pos_typ(mem, n_mem, src, dst, typ_pred)
        
#         print(logit)
        print('*' * 75)

 Did M3 's cinematographer and writer edit and executive produce M0 , M1 , and M2 
Did NP [VP_SIMPLE] NP {NIL}
[entity] 's [ROLE_SIMPLE]
[entity]
***************************************************************************
?x0 ns:film.cinematographer.film M3 .
?x0 ns:film.editor.film M0 .
?x0 ns:film.editor.film M1 .
?x0 ns:film.editor.film M2 .
?x0 ns:film.producer.films_executive_produced M0 .
?x0 ns:film.producer.films_executive_produced M1 .
?x0 ns:film.producer.films_executive_produced M2 .
?x0 ns:film.writer.film M3
***************************************************************************
?x0 ns:film.cinematographer.film M3
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.producer.films_executive_produced M0
?x0 ns:film.producer.films_executive_produced M1
?x0 ns:film.producer.films_executive_produced M2
?x0 ns:film.writer.film M3
***************************************************************************
?x0 ns:film.editor.film M0
?x0

 Did M4 's male cinematographer write , edit , and direct M0 , M1 , and M2 
Did NP [VP_SIMPLE] NP {NIL}
[entity] 's [ROLE_SIMPLE]
[entity]
***************************************************************************
?x0 ns:film.cinematographer.film M4 .
?x0 ns:film.director.film M0 .
?x0 ns:film.director.film M1 .
?x0 ns:film.director.film M2 .
?x0 ns:film.editor.film M0 .
?x0 ns:film.editor.film M1 .
?x0 ns:film.editor.film M2 .
?x0 ns:film.writer.film M0 .
?x0 ns:film.writer.film M1 .
?x0 ns:film.writer.film M2 .
***************************************************************************
?x0 ns:film.cinematographer.film M4
?x0 ns:film.director.film M0
?x0 ns:film.director.film M1
?x0 ns:film.director.film M2
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.writer.film M0
?x0 ns:film.writer.film M1
?x0 ns:film.writer.film M2
***************************************************************************
?x0 ns:film.director.film M0
?x0 ns:film.di

***************************************************************************
?x0 ns:film.director.film M0
?x0 ns:film.director.film M1
?x0 ns:film.director.film M2
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.writer.film M0
?x0 ns:film.writer.film M1
?x0 ns:film.writer.film M2
?x0 ns:film.writer.film M3
***************************************************************************
?x0 ns:film.director.film M0
?x0 ns:film.director.film M1
?x0 ns:film.director.film M2
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.writer.film M0
?x0 ns:film.writer.film M1
?x0 ns:film.writer.film M2
***************************************************************************
 Did M5 's star edit and write M0 , M1 , M2 , M3 , and M4 
Did NP [VP_SIMPLE] NP {NIL}
[entity] 's [ROLE_SIMPLE]
[entity]
***************************************************************************
?x0 ns:film.actor.film/ns:film.performance.film M

***************************************************************************
?x0 ns:film.cinematographer.film M3
?x0 ns:film.director.film M0
?x0 ns:film.director.film M1
?x0 ns:film.director.film M2
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.producer.films_executive_produced M0
?x0 ns:film.producer.films_executive_produced M1
?x0 ns:film.producer.films_executive_produced M2
?x0 ns:film.producer.film|ns:film.production_company.films M0
?x0 ns:film.producer.film|ns:film.production_company.films M1
?x0 ns:film.producer.film|ns:film.production_company.films M2
***************************************************************************
?x0 ns:film.director.film M0
?x0 ns:film.director.film M1
?x0 ns:film.director.film M2
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.producer.films_executive_produced M0
?x0 ns:film.producer.films_executive_produced M1
?x0 ns:film.producer.films_executive_produced

***************************************************************************
?x0 ns:film.cinematographer.film M4
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.writer.film M0
?x0 ns:film.writer.film M1
?x0 ns:film.writer.film M2
***************************************************************************
?x0 ns:film.editor.film M0
?x0 ns:film.editor.film M1
?x0 ns:film.editor.film M2
?x0 ns:film.writer.film M0
?x0 ns:film.writer.film M1
?x0 ns:film.writer.film M2
***************************************************************************
 Did M0 's Japanese writer write M1 , M2 , M3 , M4 , and M5 and executive produce M6 
Did NP [VP_SIMPLE] NP and [VP_SIMPLE] NP {NIL}
[entity] 's [ROLE_SIMPLE]
[entity]
[entity]
***************************************************************************
?x0 ns:film.producer.films_executive_produced M6 .
?x0 ns:film.writer.film M0 .
?x0 ns:film.writer.film M1 .
?x0 ns:film.writer.film M2 .
?x0 ns:film.writer.fi

...

***************************************************************************
?x0 ns:film.producer.films_executive_produced M1
?x0 ns:film.producer.films_executive_produced M2
?x0 ns:film.writer.film M1
?x0 ns:film.writer.film M2
?x0 ns:people.person.sibling_s/ns:people.sibling_relationship.sibling|ns:fictional_universe.fictional_character.siblings/ns:fictional_universe.sibling_relationship_of_fictional_characters.siblings M0
***************************************************************************
?x0 ns:film.producer.films_executive_produced M1
?x0 ns:film.producer.films_executive_produced M2
?x0 ns:film.writer.film M1
?x0 ns:film.writer.film M2
***************************************************************************
 Did M6 's French director influence M0 , M1 , M2 , and M3 and influence M4 
Did NP [VP_SIMPLE] NP and [VP_SIMPLE] NP {NIL}
[entity] 's [ROLE_SIMPLE]
[entity]
[entity]
***************************************************************************
?x0 ns:film.director.fil

***************************************************************************
?x0 ns:organization.organization_founder.organizations_founded M1
?x0 ns:organization.organization_founder.organizations_founded M2
?x0 ns:organization.organization_founder.organizations_founded M3
?x0 ns:organization.organization_founder.organizations_founded M4
?x0 ns:organization.organization_founder.organizations_founded M5
?x0 ns:people.person.sibling_s/ns:people.sibling_relationship.sibling|ns:fictional_universe.fictional_character.siblings/ns:fictional_universe.sibling_relationship_of_fictional_characters.siblings M0
***************************************************************************
?x0 ns:organization.organization_founder.organizations_founded M1
?x0 ns:organization.organization_founder.organizations_founded M2
?x0 ns:organization.organization_founder.organizations_founded M3
?x0 ns:organization.organization_founder.organizations_founded M4
?x0 ns:organization.organization_founder.organizations

***************************************************************************
?x0 ns:film.actor.film/ns:film.performance.character M2
?x0 ns:people.person.sibling_s/ns:people.sibling_relationship.sibling|ns:fictional_universe.fictional_character.siblings/ns:fictional_universe.sibling_relationship_of_fictional_characters.siblings M0
***************************************************************************

***************************************************************************
 Did M2 's Spanish director marry M1 
Did NP [VP_SIMPLE] NP {NIL}
[entity] 's [ROLE_SIMPLE]
[entity]
***************************************************************************
?x0 ns:film.director.film M2 .
?x0 ns:people.person.spouse_s/ns:people.marriage.spouse|ns:fictional_universe.fictional_character.married_to/ns:fictional_universe.marriage_of_fictional_characters.spouses M1 .
***************************************************************************
?x0 ns:film.director.film M2
?x0 ns:people.person.spou

***************************************************************************
?x0 ns:film.editor.film M2
?x0 ns:film.producer.films_executive_produced M0
***************************************************************************
?x0 ns:film.producer.films_executive_produced M0
***************************************************************************
 Did M3 's executive producer edit , direct , write , and produce M0 , M1 , and M2 
Did NP [VP_SIMPLE] NP {NIL}
[entity] 's [ROLE_SIMPLE]
[entity]
***************************************************************************
?x0 ns:film.director.film M0 .
?x0 ns:film.director.film M1 .
?x0 ns:film.director.film M2 .
?x0 ns:film.editor.film M0 .
?x0 ns:film.editor.film M1 .
?x0 ns:film.editor.film M2 .
?x0 ns:film.producer.films_executive_produced M3 .
?x0 ns:film.producer.film|ns:film.production_company.films M0 .
?x0 ns:film.producer.film|ns:film.production_company.films M1 .
?x0 ns:film.producer.film|ns:film.production_company.films M2 .
?

***************************************************************************
M0 ns:film.director.film M1
M0 ns:film.director.film M2
M0 ns:film.director.film M3
M0 ns:film.editor.film M1
M0 ns:film.editor.film M2
M0 ns:film.editor.film M3
M0 ns:film.producer.films_executive_produced M1
M0 ns:film.producer.films_executive_produced M2
M0 ns:film.producer.films_executive_produced M3
M0 ns:film.producer.film|ns:film.production_company.films M1
M0 ns:film.producer.film|ns:film.production_company.films M2
M0 ns:film.producer.film|ns:film.production_company.films M3
M0 ns:film.writer.film M1
M0 ns:film.writer.film M2
M0 ns:film.writer.film M3
***************************************************************************
M0 ns:film.director.film M1
M0 ns:film.director.film M2
M0 ns:film.director.film M3
M0 ns:film.editor.film M1
M0 ns:film.editor.film M2
M0 ns:film.editor.film M3
M0 ns:film.producer.films_executive_produced M1
M0 ns:film.producer.films_executive_produced M2
M0 ns:film.producer.fil

In [178]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [196]:
split = 'mcd3'
dat = np.load(f'{input_dir}/data.npz')
idx_train = np.load(f'{input_dir}/splits/{split}.npz')['trainIdxs']
ds_train = CFQDataset(idx_train, dat, tok_vocab, tag_vocab, typ_vocab)
get_train = lambda key: ds_train[np.where(idx_train == key)[0].item()]
idx_test = np.load(f'{input_dir}/splits/{split}.npz')['testIdxs']
ds_test = CFQDataset(idx_test, dat, tok_vocab, tag_vocab, typ_vocab)
get_test = lambda key: ds_test[np.where(idx_test == key)[0].item()]
to_str = lambda templ: ' '.join(idx2tag[idx] for idx in templ)

In [197]:
templ_train = [to_str(get_train(index)['seq_tag']) for index in idx_train]
templ_test = [to_str(get_test(index)['seq_tag']) for index in idx_test]
templ_train_uniq, templ_test_uniq = set(templ_train), set(templ_test)

In [198]:
n = lambda templ: sum(1 for i, em in zip(out['index'], out['em']) if seq2str(getitem(i)['seq_tag']) == templ)
n_em = lambda templ: sum(1 for i, em in zip(out['index'], out['em']) if em and seq2str(getitem(i)['seq_tag']) == templ)

In [203]:
templ_top = sorted(([templ, templ_test.count(templ)] for templ in templ_test_uniq), key=lambda x: x[1], reverse=True)
templ_top[:10]

[['Was NP NP {NIL}', 2527],
 ['Did NP [VP_SIMPLE] NP {NIL}', 2449],
 ['NP was NP {NIL}', 1400],
 ['Was NP [VP_SIMPLE] by NP {NIL}', 987],
 ['NP did NP [VP_SIMPLE] {NIL}', 526],
 ['Was NP [VP_SIMPLE] by NP and [VP_SIMPLE] by NP {NIL}', 449],
 ['NP [VP_SIMPLE] NP {NIL}', 422],
 ['Was NP [VP_SIMPLE] by NP , [VP_SIMPLE] by NP , and [VP_SIMPLE] by NP {NIL}',
  383],
 ['Did NP [VP_SIMPLE] NP and [VP_SIMPLE] NP {NIL}', 340],
 ['NP was [VP_SIMPLE] by NP {NIL}', 319]]

In [206]:
sum(n for _, n in templ_top[:10]) / len(templ_test)

0.8190173796791443

In [201]:
for templ, _ in templ_top[:10]:
    print(templ, t_train.count(templ), n_em(templ) / n(templ))

Was NP NP {NIL} 11617 0.5112781954887218
Did NP [VP_SIMPLE] NP {NIL} 5609 0.18456512862392813
NP was NP {NIL} 1719 0.9335714285714286
Was NP [VP_SIMPLE] by NP {NIL} 5047 0.3525835866261398
NP did NP [VP_SIMPLE] {NIL} 6852 1.0
Was NP [VP_SIMPLE] by NP and [VP_SIMPLE] by NP {NIL} 2849 0.2561247216035635
NP [VP_SIMPLE] NP {NIL} 4397 1.0
Was NP [VP_SIMPLE] by NP , [VP_SIMPLE] by NP , and [VP_SIMPLE] by NP {NIL} 1454 0.4281984334203655
Did NP [VP_SIMPLE] NP and [VP_SIMPLE] NP {NIL} 3286 0.13529411764705881
NP was [VP_SIMPLE] by NP {NIL} 4590 1.0


In [160]:
for i in idx_train[:1000]:
    dat = get_train(i)
    if to_str(dat['seq_tag']) == 'Did NP [VP_SIMPLE] NP {NIL}':
        print(question[i])

Did M1 employ a spouse of a actor
Did M1 write , direct , edit , and produce a sequel of M0
Did M1 , M2 , M3 , and M4 acquire a child of M0
Did M1 executive produce , write , direct , edit , and produce M0 's sequel
Did M1 acquire a production company 's parent
Did M2 and M3 marry a French film director 's spouse
Did M0 and M2 executive produce and write M1
Did M2 and M3 employ a British screenwriter
Did M1 produce a film 's sequel and prequel
Did M3 influence a French German actor 's spouse
Did M1 , M2 , and M3 executive produce , direct , produce , and write a film
Did M1 , M2 , and M3 executive produce M0 's prequel
Did M2 influence a female spouse of M1 's director
Did M1 influence M0 's actor
Did M1 , M2 , M3 , and M4 marry a cinematographer 's child
Did M3 marry a male Spanish screenwriter 's child
Did M1 influence and marry a person
Did M1 star a actor 's actor
Did M2 marry a Chinese actor
Did M0 , M2 , and M3 write and executive produce M1
Did M1 , M2 , M3 , and M4 influence M0

In [114]:
templ_train1, templ_test1 = templ_train, templ_test

In [117]:
templ_train3, templ_test3 = templ_train, templ_test

In [118]:
len(templ_train1), len(templ_train3), len(templ_train1.intersection(templ_train3))

(1205, 934, 866)

In [119]:
len(templ_test1), len(templ_test3), len(templ_test1.intersection(templ_test3))

(323, 317, 229)

In [128]:
both = templ_train.intersection(templ_test)
print(len(templ_train), len(templ_test), len(both))
list(islice(both, 10))

934 317 150


['Was NP that [VP_SIMPLE] NP and was [VP_SIMPLE] by NP NP {NIL}',
 'NP that NP [VP_SIMPLE] was NP {NIL}',
 'Was NP NP that NP was [VP_SIMPLE] by and [VP_SIMPLE] {NIL}',
 'NP was [VP_SIMPLE] by NP and [VP_SIMPLE] NP {NIL}',
 'Was NP [VP_SIMPLE] by NP that NP [VP_SIMPLE] {NIL}',
 'NP [VP_SIMPLE] by NP [VP_SIMPLE] and was [VP_SIMPLE] by NP {NIL}',
 'NP that was [VP_SIMPLE] by NP did NP [VP_SIMPLE] {NIL}',
 'Was NP NP that [VP_SIMPLE] , [VP_SIMPLE] , and was [VP_SIMPLE] by NP {NIL}',
 'Did NP [VP_SIMPLE] NP , [VP_SIMPLE] NP , and [VP_SIMPLE] NP {NIL}',
 'NP [VP_SIMPLE] NP , [VP_SIMPLE] NP , and [VP_SIMPLE] NP {NIL}']

In [13]:
sum(1 for i in idx_test if seq2str(get_test(i)['seq_tag']) in templ_train)

NameError: name 'idx_test' is not defined

In [113]:
templ_test.difference(templ_train)

{'Did NP [VP_SIMPLE] NP , [VP_SIMPLE] NP , [VP_SIMPLE] NP , and [VP_SIMPLE] NP {NIL}',
 'Did NP that NP [VP_SIMPLE] , was [VP_SIMPLE] by , was [VP_SIMPLE] by , and was [VP_SIMPLE] by [VP_SIMPLE] NP {NIL}',
 'Did NP that NP [VP_SIMPLE] [VP_SIMPLE] NP , [VP_SIMPLE] NP , and [VP_SIMPLE] NP {NIL}',
 'Did NP that NP [VP_SIMPLE] and NP were [VP_SIMPLE] by [VP_SIMPLE] NP {NIL}',
 'Did NP that NP [VP_SIMPLE] and were [VP_SIMPLE] by [VP_SIMPLE] NP {NIL}',
 'Did NP that NP was [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , and [VP_SIMPLE] by [VP_SIMPLE] NP {NIL}',
 'Did NP that NP were [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , and [VP_SIMPLE] by [VP_SIMPLE] NP {NIL}',
 'Did NP that NP were [VP_SIMPLE] by , [VP_SIMPLE] by , and [VP_SIMPLE] by [VP_SIMPLE] NP {NIL}',
 'Did NP that NP were [VP_SIMPLE] by and NP were [VP_SIMPLE] by [VP_SIMPLE] NP {NIL}',
 'Did NP that NP were [VP_SIMPLE] by and [VP_SIMPLE] [VP_SIMPLE] NP and [VP_SIMPLE] NP {NIL}',
 'Did NP that NP were [VP_SIMPLE] by and

In [130]:
from functools import *

In [210]:
np_train = reduce(set.union, (set(map(tuple, get_train(i)['seq_noun'])) for i in idx_train))
np_test = reduce(set.union, (set(map(tuple, get_test(i)['seq_noun'])) for i in idx_test))

In [217]:
sorted(to_str(np) for np in np_test)

['What',
 'What [NP_SIMPLE]',
 'What [NP_SIMPLE] whose [ROLE_SIMPLE]',
 'What [ROLE_SIMPLE] of [entity]',
 'What [ROLE_SIMPLE] of a [NP_SIMPLE]',
 'Which [NP_SIMPLE]',
 'Which [NP_SIMPLE] whose [ROLE_SIMPLE]',
 'Which [ROLE_SIMPLE] of [entity]',
 'Which [ROLE_SIMPLE] of a [NP_SIMPLE]',
 'Who',
 '[entity]',
 "[entity] 's [ROLE_SIMPLE]",
 "[entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE]",
 'a [NP_SIMPLE]',
 "a [NP_SIMPLE] 's [ROLE_SIMPLE]",
 "a [NP_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE]",
 'a [NP_SIMPLE] whose [ROLE_SIMPLE]',
 'a [ROLE_SIMPLE] of [entity]',
 "a [ROLE_SIMPLE] of [entity] 's [ROLE_SIMPLE]",
 'a [ROLE_SIMPLE] of a [NP_SIMPLE]',
 'a [ROLE_SIMPLE] of a [NP_SIMPLE] whose [ROLE_SIMPLE]',
 'a [ROLE_SIMPLE] of a [ROLE_SIMPLE] of [entity]']

In [213]:
len(np_train), len(np_test), len(np_train.intersection(np_test))

(56, 22, 22)

In [136]:
[to_str(np) for np in np_test.difference(np_train)]

['Which [ROLE_SIMPLE] of [entity]',
 'What [ROLE_SIMPLE] of [entity]',
 'What [ROLE_SIMPLE] of a [NP_SIMPLE]',
 'Which [ROLE_SIMPLE] of a [NP_SIMPLE]']

In [152]:
sorted(' '.join(idx2tag[idx] for idx in np) for np in np_train)

['What',
 'What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE] whose [ROLE_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 'What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [NP_SIMPLE] whose [ADJECTIVE_SIMPLE] [ROLE_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [NP_SIMPLE] whose [ROLE_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 "What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] 's [ROLE_SIMPLE]",
 'What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [ADJECTIVE_SIMPLE] [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [NP_SIMPLE]',
 "What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [NP_SIMPLE] 's [ROLE_SIMPLE]",
 'What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [ROLE_SIMPLE] of [entity]',
 'What [NP_SIMPLE]',
 'What [NP_SI

In [143]:
[' '.join(idx2tag[idx] for idx in np) for np in np_test]

['a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 "a [NP_SIMPLE] 's [ROLE_SIMPLE]",
 "a [NP_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE]",
 'Which [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 'a [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 'a [ADJECTIVE_SIMPLE] [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 'Which [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE]',
 '[entity]',
 'Which [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [NP_SIMPLE]',
 "[entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE]",
 'Which [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 'What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [NP_SIMPLE]',
 "[entity] 's [ROLE_SIMPLE]",
 'Which [ROLE_SIMPLE] of a [NP_SIMPLE]',
 'What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity]',
 "a [ADJECTIVE_SIMPLE] [NP_SIMPLE] 's [ROLE_SIMPLE]",
 'What [NP_SIMPLE

In [151]:
for np_ in np_test:
    np__ = ' '.join(idx2tag[idx] for idx in np_)
    print(bcolors.B if np_ in np_train else bcolors.R, np__, bcolors.ENDC)

 a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] 
 a [NP_SIMPLE] 's [ROLE_SIMPLE] 
 a [ROLE_SIMPLE] of a [NP_SIMPLE] whose [ROLE_SIMPLE] 
 a [NP_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 
 Which [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] 
 a [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] 
 a [ADJECTIVE_SIMPLE] [NP_SIMPLE] 
 What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] 
 Which [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE] 
 [entity] 
 [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 
 Which [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] 
 What [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE] 
 What [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of a [NP_SIMPLE] 
 [entity] 's [ROLE_SIMPLE] 
 Which [ROLE_SIMPLE] of a [NP_SIMPLE] 
 a [ADJECTIVE_SIMPLE] [NP_SIMPLE] 's [ROLE_SIMPLE] 
 What [NP_SIMPLE] 
 a [NP_SIMPLE] whose [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 
 a [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [NP_SIMPLE] 

In [84]:
len(np_train), len(np_test), len(np_train.intersection(np_test))

(351, 53, 52)

In [268]:
import re
from operator import *

In [271]:
def n_var(r):
    return len(set(re.findall(r'\?x\d', r['sparqlPatternModEntities'])))

def n(r):
    return sum(1 for tok in r['questionTemplate'].split(' ') if tok in ['[NP_SIMPLE]', '[ROLE_SIMPLE]', 'Who'])

# n_var(df.rdd.take(1)[0])
# n(df.rdd.take(1)[0])
df.rdd.map(lambda r: n_var(r) == n(r)).reduce(add)

115974

In [273]:
df.rdd.filter(lambda r: n_var(r) != n(r)).map(lambda r: r['questionTemplate']).take(10)

["Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [entity] and [VP_SIMPLE] a [NP_SIMPLE] 's [ROLE_SIMPLE]",
 "Did [entity] [VP_SIMPLE] [entity] 's [ROLE_SIMPLE] , [VP_SIMPLE] a [ROLE_SIMPLE] and [ROLE_SIMPLE] of [entity] and [entity] , and

In [ ]:
df.rdd.map(lambda r: r['questionTemplate']).filter(lambda r: r.startswith(''))

In [270]:
df.count()

239357

In [261]:
print(df.rdd.map(lambda r: r['sparqlPatternModEntities']).take(1)[0])

SELECT count(*) WHERE {
?x0 ns:film.actor.film/ns:film.performance.character M1 .
?x0 ns:film.editor.film M0 .
?x0 ns:film.producer.film|ns:film.production_company.films M0 .
?x0 ns:people.person.gender ns:m.02zsn
}


In [255]:
df.columns

['complexityMeasures',
 'expectedResponse',
 'expectedResponseWithMids',
 'index',
 'question',
 'questionPatternModEntities',
 'questionTemplate',
 'questionWithBrackets',
 'questionWithMids',
 'ruleIds',
 'ruleTree',
 'sparql',
 'sparqlPattern',
 'sparqlPatternModEntities']

In [220]:
sum(1 for index in idx_test if ' '.join([idx2tag[idx] for idx in get_test(index)['seq']]) in both)

3081

In [247]:
[x for x in templ_train if x.startswith("Was [entity] 's")]

["Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 ?x2 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 ?x2 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Was [entity] 's [

In [249]:
[x for x in templ_train if "a [NP_SIMPLE]" in x]

["Who was [VP_SIMPLE] by [entity] 's [ROLE_SIMPLE] and [VP_SIMPLE] by a [ROLE_SIMPLE] of a [NP_SIMPLE] {SEP} ?x0 ?x1 ?x2 ?x3 {NIL}",
 'What was [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , and [VP_SIMPLE] by a [NP_SIMPLE] that [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}',
 'What was [VP_SIMPLE] by a [NP_SIMPLE] that [entity] was [VP_SIMPLE] by and [VP_SIMPLE] by and [VP_SIMPLE] by [entity] {SEP} ?x0 ?x1 {NIL}',
 'Which [NP_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] that [entity] was [VP_SIMPLE] by and [entity] [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}',
 'Did [entity] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] , and [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}',
 "Was [entity] a [NP_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] {SEP} ?x0 ?x1 ?x2 {NIL}",
 'Who was [VP_SIMPLE] by a [NP_SIMPLE] and was [VP_SIMPLE] by a [ROLE_SIMPLE] of a [ROLE_SIMPLE] of [entity] {SEP} ?x0 ?x1 ?x2 ?x3 {NIL}',
 'What [NP_SIMP

In [241]:
templ_test.difference(templ_train)

{"Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] [entity] , and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [NP_SIMPLE] , and [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP

In [243]:
set.difference?